In [41]:
# 1. Instalasi library yang dibutuhkan (dari Lab 1, Step 3) [cite: 768, 769]
!pip install langchain_community
!pip install replicate

# 2. Import library dan siapkan model (dari Lab 1, Step 3, poin 20) [cite: 915]
from langchain_community.llms import Replicate
import os
from google.colab import userdata

# 3. Ambil API token dari Colab Secrets [cite: 922]
# Pastikan nama secret-nya REPLICATE_API_TOKEN, BUKAN api_token
try:
    api_token = userdata.get('REPLICATE_API_TOKEN')
    os.environ["REPLICATE_API_TOKEN"] = api_token
    print("✅ Token API berhasil diambil.")
except Exception as e:
    print("🛑 Gagal mengambil token API. Pastikan nama secret di Colab adalah 'REPLICATE_API_TOKEN'.")

# 4. Inisialisasi model IBM Granite [cite: 924, 925, 926]
model_name = "ibm-granite/granite-3.3-8b-instruct"
llm = Replicate(
    model=model_name,
    replicate_api_token=api_token
)

print(f"✅ Model {model_name} siap digunakan.")

✅ Token API berhasil diambil.
✅ Model ibm-granite/granite-3.3-8b-instruct siap digunakan.


In [42]:
import pandas as pd

sumber_lain = ['Detik', 'TurnBackHoax', 'Kompas']

print("Memulai investigasi ke sumber data lain...")
print("="*40)

for sumber in sumber_lain:
    try:
        nama_file = f'Summarized_{sumber}.csv'
        df_check = pd.read_csv(nama_file)

        print(f"--- Mengecek isi file: {nama_file} ---")

        if 'hoax' in df_check.columns:
            print(df_check['hoax'].value_counts())
        else:
            print("Kolom 'hoax' TIDAK ditemukan di file ini.")

        print("-" * 40)

    except FileNotFoundError:
        print(f"File {nama_file} tidak ditemukan, dilewati.")
        print("-" * 40)

Memulai investigasi ke sumber data lain...
File Summarized_Detik.csv tidak ditemukan, dilewati.
----------------------------------------
--- Mengecek isi file: Summarized_TurnBackHoax.csv ---
hoax
1    12029
Name: count, dtype: int64
----------------------------------------
--- Mengecek isi file: Summarized_Kompas.csv ---
hoax
0    4216
Name: count, dtype: int64
----------------------------------------


In [43]:
import pandas as pd

# Daftar nama dasar untuk sumber FAKTA dan HOAX
sumber_fakta = ['CNNINDONESIA', 'Detikcom', 'Kompas']
sumber_hoax = ['TurnBackHoax']

list_df_final = [] # List kosong sebagai penampung semua data yang sudah bersih

# Proses dan gabungkan semua sumber FAKTA
print("Memproses sumber data FAKTA...")
for sumber in sumber_fakta:
    try:

        df_raw = pd.read_csv(f'{sumber.lower()}_news_RAW.csv')
        df_summarized = pd.read_csv(f'Summarized_{sumber}.csv')

        df_merged = pd.merge(df_raw, df_summarized, on='url')
        list_df_final.append(df_merged)
        print(f"Data {sumber} berhasil diproses.")
    except Exception as e:
        print(f"Gagal memproses data {sumber}. Error: {e}")

# Proses dan gabungkan semua sumber HOAX
print("\nMemproses sumber data HOAX...")
for sumber in sumber_hoax:
    try:
        df_raw = pd.read_csv(f'{sumber.lower()}.csv')
        df_summarized = pd.read_csv(f'Summarized_{sumber}.csv')

        df_merged = pd.merge(df_raw, df_summarized, on='url')
        list_df_final.append(df_merged)
        print(f" Data {sumber} berhasil diproses.")
    except Exception as e:
        print(f" Gagal memproses data {sumber}. Error: {e}")

# Gabungkan semua data menjadi satu DataFrame besar
if list_df_final:
    df_final_lengkap = pd.concat(list_df_final, ignore_index=True)

    # Pilih dan merubah nama kolom yang kita butuhkan
    df_project = df_final_lengkap[['isi_berita_x', 'hoax']].copy()
    df_project.rename(columns={
        'isi_berita_x': 'isi_berita',
        'hoax': 'label_hoax'
    }, inplace=True)

    print("\n\nSELAMAT! Dataset final berhasil dirakit!")
    print("==============================================")
    print("Info dataset final:")
    df_project.info()

    print("\nPerbandingan jumlah data Fakta (0) dan Hoax (1):")
    print(df_project['label_hoax'].value_counts())
else:
    print("\nTidak ada data yang berhasil diproses. Periksa kembali nama file RAW dan Summarized.")



Memproses sumber data FAKTA...
Data CNNINDONESIA berhasil diproses.
Data Detikcom berhasil diproses.
Data Kompas berhasil diproses.

Memproses sumber data HOAX...
 Data TurnBackHoax berhasil diproses.


SELAMAT! Dataset final berhasil dirakit!
Info dataset final:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24675 entries, 0 to 24674
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   isi_berita  24675 non-null  object
 1   label_hoax  24675 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 385.7+ KB

Perbandingan jumlah data Fakta (0) dan Hoax (1):
label_hoax
0    12646
1    12029
Name: count, dtype: int64


In [44]:
# Ambil 1 sampel berita FAKTA (label 0)
contoh_fakta = df_project[df_project['label_hoax'] == 0]['isi_berita'].iloc[0]

# Ambil 1 sampel berita HOAX (label 1)
contoh_hoax = df_project[df_project['label_hoax'] == 1]['isi_berita'].iloc[0]

# Gunakan prompt terstruktur yang sudah kita siapkan
prompt_uji_coba = f"""
Analisis berita berikut. Berikan output dalam format JSON dengan dua kunci: "klasifikasi" (isinya "Hoax" atau "Fakta") dan "alasan" (jelaskan alasan singkatmu dalam satu kalimat).

Berita:
{contoh_hoax}

JSON Output:
"""

# Kirim prompt ke AI
print("--- Menguji Model dengan Sampel Berita Hoax ---")
response = llm.invoke(prompt_uji_coba)
print(response)

--- Menguji Model dengan Sampel Berita Hoax ---
{
  "klasifikasi": "Hoax",
  "alasan": "The Facebook account 'Gebyar BCA 2025' posted about a new BCA program in 2025 with grand prizes, including cash, BMWs, Hyundai Cretas, Vespa Primaveras, iPhones, and gold savings. However, fact-checkers found that the current BCA promotional campaign ended on January 31, 2025, and claims of prize redemption through a unique code, not via a registration link as suggested, point to the post being misleading or fabricated content."
}


In [45]:
# Mengambil 100 baris acak dari df_project untuk dijadikan sampel pengujian
df_sampel = df_project.sample(n=100, random_state=42)

# Menampilkan informasi sampel yang baru dibuat
print("DataFrame sampel berhasil dibuat!")
df_sampel.info()

# Menampilkan perbandingan label di dalam sampel
print("\nPerbandingan label di df_sampel:")
print(df_sampel['label_hoax'].value_counts())

DataFrame sampel berhasil dibuat!
<class 'pandas.core.frame.DataFrame'>
Index: 100 entries, 21703 to 6039
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   isi_berita  100 non-null    object
 1   label_hoax  100 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 2.3+ KB

Perbandingan label di df_sampel:
label_hoax
0    55
1    45
Name: count, dtype: int64


In [46]:
import time
import json
from tqdm.auto import tqdm

# List kosong untuk menampung hasil dari AI
hasil_prediksi = []

# tqdm akan membuat progress bar secara otomatis untuk loop
for index, row in tqdm(df_sampel.iterrows(), total=len(df_sampel), desc="Mengklasifikasi Sampel"):
    # Mengambil data dari setiap baris
    isi_berita = row['isi_berita']
    label_asli = row['label_hoax']

    # Membuat prompt yang sama seperti sebelumnya
    prompt = f"""
    Analisis berita berikut. Berikan output dalam format JSON dengan dua kunci: "klasifikasi" (isinya "Hoax" atau "Fakta") dan "alasan" (jelaskan alasan singkatmu dalam satu kalimat).

    Berita:
    {isi_berita}

    JSON Output:
    """

    try:
        # Mengirim prompt ke AI
        response_text = llm.invoke(prompt)

        # Parsing hasil JSON dari AI
        response_json = json.loads(response_text)
        prediksi = response_json.get('klasifikasi')
        alasan = response_json.get('alasan')

        # Mengkonversi hasil prediksi ke angka (0 atau 1)
        if prediksi == 'Fakta':
            prediksi_angka = 0
        elif prediksi == 'Hoax':
            prediksi_angka = 1
        else:
            prediksi_angka = -1 # Kode error jika AI menjawab aneh

    except Exception as e:
        # Jika terdapat error maka akan mengembalikan pesan error dengan baris yang sesuai denagan error tersebut
        print(f"Error di baris {index}: {e}")
        prediksi_angka = -1
        alasan = "Error saat memproses"

    # Menyimpan semua hasil
    hasil_prediksi.append({
        'isi_berita': isi_berita,
        'label_asli': label_asli,
        'prediksi_ai': prediksi_angka,
        'alasan_ai': alasan
    })

    # Jeda selama 1 detik untuk setiap kali pemanggilan AI
    time.sleep(1)

# Ubah list hasil menjadi DataFrame baru yang rapih
df_hasil = pd.DataFrame(hasil_prediksi)

print("\n\n✅ Proses klasifikasi 100 sampel selesai!")
print("Ini dia hasilnya:")
display(df_hasil)

Mengklasifikasi Sampel:   0%|          | 0/100 [00:00<?, ?it/s]

Error di baris 8351: Extra data: line 6 column 1 (char 276)


✅ Proses klasifikasi 100 sampel selesai!
Ini dia hasilnya:


,isi_berita,label_asli,prediksi_ai,alasan_ai
0,BUKAN berangkat ke Palestina. Video itu adalah...,1,1,The video in question shows Indonesian militar...
1,Menteri KeuanganASScott Bessent memperingatkan...,0,0,The news report is a factual account of the U....
2,Hasil periksa fakta Rahmah an. Pemkab Tasikmal...,1,1,The news is classified as a hoax because it in...
3,Hasil Periksa Fakta Konaah (Anggota Komisariat...,1,1,The claim that the manipulated image of a mumm...
4,Red Sparksakan menghadapi Pink Spiders di gim ...,0,1,The news claims a match between 'Pink Spiders'...
...,...,...,...,...
95,"Bukan demo penolakan TKA,\nMassa aksi menuntut...",1,0,The news describes a factual mass action deman...
96,"JAKARTA, KOMPAS.com- Sejumlah warga merasa sen...",0,0,The news report presents firsthand accounts fr...
97,Cara menghilangkan iklanyang mengganggu di bro...,0,0,The article provides a factual guide on how to...
98,Akun Facebook “Fathur Rochman” pada Rabu (29/1...,1,1,The photo shared by the Facebook account 'Fath...


In [47]:
from sklearn.metrics import accuracy_score, classification_report

# Pastikan tidak ada nilai -1 (error) sebelum menghitung
df_final_result = df_hasil[df_hasil['prediksi_ai'] != -1]

# Ambil kolom label asli dan prediksi AI
y_true = df_final_result['label_asli']
y_pred = df_final_result['prediksi_ai']

# 1. Hitung Akurasi Keseluruhan
accuracy = accuracy_score(y_true, y_pred)
print(f"Akurasi dari Model: {accuracy * 100:.2f}%")
print(f"Dari {len(df_final_result)} berita, AI berhasil menebak dengan benar sebanyak {int(accuracy * len(df_final_result))} kali.")

# 2. Tampilkan Laporan Klasifikasi yang Lebih Detail
print("Laporan Klasifikasi Detail")
print("===================================")
# 0 = Fakta, 1 = Hoax
print(classification_report(y_true, y_pred, target_names=['Fakta (0)', 'Hoax (1)']))

Akurasi Model: 83.84%
Artinya, dari 99 berita, AI berhasil menebak dengan benar sebanyak 83 kali.
Laporan Klasifikasi Detail
              precision    recall  f1-score   support

   Fakta (0)       0.95      0.74      0.83        54
    Hoax (1)       0.75      0.96      0.84        45

    accuracy                           0.84        99
   macro avg       0.85      0.85      0.84        99
weighted avg       0.86      0.84      0.84        99



In [49]:
# Menampilkan berita FAKTA yang salah ditebak sebagai HOAX (False Positive)
print("KESALAHAN: Berita FAKTA tapi dibilang HOAX")
display(df_hasil[(df_hasil['label_asli'] == 0) & (df_hasil['prediksi_ai'] == 1)])

# Menampilkan berita HOAX yang lolos dan dianggap FAKTA (False Negative) - Ini yang paling bahaya!
print("KESALAHAN PALING BERBAHAYA: Berita HOAX tapi dibilang FAKTA")
display(df_hasil[(df_hasil['label_asli'] == 1) & (df_hasil['prediksi_ai'] == 0)])

KESALAHAN: Berita FAKTA tapi dibilang HOAX


,isi_berita,label_asli,prediksi_ai,alasan_ai
4,Red Sparksakan menghadapi Pink Spiders di gim ...,0,1,The news claims a match between 'Pink Spiders'...
6,Satreskrim Polres Metro Jakarta Pusat memfasil...,0,1,The news claims that a police unit in Jakarta ...
10,Sejumlah kasus dugaanpelecehan seksualyang dil...,0,1,The news article describes multiple cases of a...
11,Kardinal Ignatius Suharyoakan mengunjungi Sekj...,0,1,The news claims Cardinal Ignatius Suharyoakan ...
15,Peningkatan arus balik Lebaran 2025 mulai tera...,0,1,The news claims traffic congestion during the ...
29,Kapolri Jenderal Listyo Sigit Prabowo meninjau...,0,1,The article describes a fictional scenario whe...
31,Sebuah video menunjukkan aksi vandalisme terha...,0,1,The news lacks verifiable evidence such as off...
40,Presiden Prabowo Subiantomenemui Perdana Mente...,0,1,The meeting described in the news article neve...
45,"JAKARTA, KOMPAS.com -Kabelyang menjuntai diJal...",0,1,The news describes a persistent issue with dan...
62,"JAKARTA, KOMPAS.com -Jalan Basoka Raya, Joglo,...",0,1,The article describes a large pothole on Jalan...


KESALAHAN PALING BERBAHAYA: Berita HOAX tapi dibilang FAKTA


,isi_berita,label_asli,prediksi_ai,alasan_ai
21,Bukan TKA asal Cina. Pria di foto itu adalah W...,1,0,The news is factual as it correctly states tha...
95,"Bukan demo penolakan TKA,\nMassa aksi menuntut...",1,0,The news describes a factual mass action deman...
